In [8]:
# external packages
from pathlib import Path
import matplotlib.pyplot as plt 
from importlib import reload
import matplotlib
%matplotlib qt
import numpy as np
matplotlib.style.use('default')
from datetime import datetime
import pandas as pd

In [9]:
# local modules and packages
from ForexMachine.Preprocessing import get_indicators as gi
from ForexMachine import util
reload(gi)
reload(util)

<module 'ForexMachine.util' from 'c:\\github repos\\forexmachine\\ForexMachine\\util.py'>

In [10]:
# convert config to dictionary
config = util.yaml_to_dict()
current_model = config['current_model']
indicators = config[current_model]['indicators']
# Read in data with indicators
data_with_indicators = gi.add_indicators_to_raw(filepath='..\Data\RawData\EURUSDi1440.csv',save_to_disk=True,config=config)
data_with_indicators.head()

FileNotFoundError: [Errno 2] File ..\ForexMachine\Data\RawData\EURUSDi1440.csv does not exist: '..\\ForexMachine\\Data\\RawData\\EURUSDi1440.csv'

In [4]:
def get_index_range(df,datetime1,datetime2):
    i1 = -1
    i2 = -1
    if datetime1 <= datetime2:
        for i in range(len(df)):
            i1 = i
            if df['datetime'][i] == datetime1:
                break
            if df['datetime'][i] > datetime1:
                i1 = i-1 if i-1 >= 0 else 0
                break
        for i in range(i1, len(df)):
            i2 = i
            if df['datetime'][i] == datetime2:
                break
            if df['datetime'][i] > datetime2:
                i2 = i-1 if i-1 >= 0 else 0
                break
    return i1, i2

# date format 'yyyy.mm.dd'
def show_data_from_range(df, date1, date2, main_indicators = [], sub_indicators = []):
    start, stop = get_index_range(df, pd.Timestamp.fromisoformat(date1), pd.Timestamp.fromisoformat(date2))
    if start < 0 or stop < 0:
        print('invalid dates')
        return
    data_range = df.iloc[start:stop+1]
    chart_count = len(sub_indicators) + 1
    
    top_chart_ratio = 1
    sub_chart_ratio = 0
    if chart_count == 2:
        top_chart_ratio = 3
        sub_chart_ratio = 2 / (chart_count-1)
    if chart_count > 2:
        top_chart_ratio = 1
        sub_chart_ratio = 1 / (chart_count-1)
    height_ratios = [top_chart_ratio]
    height_ratios.extend([sub_chart_ratio]*(chart_count-1))
    fig, axes = plt.subplots(chart_count,1,sharex='col', gridspec_kw={'height_ratios':height_ratios})
    fig.tight_layout(pad=1.8, h_pad=0.0)
    
    top_ax = None
    bottom_ax = None
    if chart_count > 1:
        top_ax = axes[0]
        bottom_ax = axes[len(axes)-1]
    else:
        bottom_ax = top_ax = axes
    top_ax.plot(data_range.Close.to_list(), label='Close',color='brown')
    
    plot_indicator_funcs = {
        'ichimoku': lambda ax, df: add_ichimoku_to_plot(ax, df),
        'rsi': lambda ax, df: add_rsi_to_plot(ax ,df)
    }
    
    for indicator in main_indicators:
        plot_indicator_funcs[indicator](top_ax, data_range)
    
    for i in range(len(sub_indicators)):
        plot_indicator_funcs[sub_indicators[i]](axes[i+1], data_range)

    bottom_ax.set_xticks(np.arange(len(data_range)))
    #     condition = data_range.index % 10 == 0
    #     condition[-1] = True
    #     x_labels = np.where(condition, data_range['Date'], None)
    x_labels = data_range['Date']
#     top_ax.axvline(stop-start-5,color='red')
    bottom_ax.set_xticklabels(x_labels,rotation=90)
    
    if chart_count > 1:
        for ax in axes:
            ax.legend()
    else:
        top_ax.legend()
    plt.show()

In [5]:
"""
Functions for adding indicators to a matplotlib chart
"""

def add_ichimoku_to_plot(ax, df):
    ax.plot(df.trend_visual_ichimoku_a.to_list(), label='Senkou-Span a',linestyle='--',color='green')
    ax.plot(df.trend_visual_ichimoku_b.to_list(), label='Senkou-Span b',linestyle='--',color='red')
    ax.fill_between(np.arange(len(df)),df.trend_visual_ichimoku_a,
                    df.trend_visual_ichimoku_b,alpha=0.2,color='green',
                    where=(df.trend_visual_ichimoku_a > df.trend_visual_ichimoku_b))
    ax.fill_between(np.arange(len(df)),df.trend_visual_ichimoku_a,
                    df.trend_visual_ichimoku_b,alpha=0.2,color='red',
                    where=(df.trend_visual_ichimoku_a <= df.trend_visual_ichimoku_b))
    ax.plot(df.trend_ichimoku_conv.to_list(), label='Tenkan-Sen (conversion)',color='cyan')
    ax.plot(df.trend_ichimoku_base.to_list(), label='Kijun Sen (base)',color='blue')
    ax.plot(df.chikou_span.to_list(), label='chikou span',linestyle=':',color='orange')

def add_rsi_to_plot(ax, df):
    ax.plot(df.momentum_rsi.to_list(), label='RSI', color='purple')
    ax.plot([30]*len(df),color='gray',alpha=0.5)
    ax.plot([70]*len(df),color='gray',alpha=0.5)
    ax.fill_between(np.arange(len(df)),[30]*len(df),[70]*len(df),color='gray',alpha=0.2)
    ax.set_ylim(15,85)
    ax.set_yticks(np.arange(20,100,20))

In [6]:
show_data_from_range(data_with_indicators, '2020-04-28T03:25', '2020-06-05', main_indicators=['ichimoku'], sub_indicators=['rsi'])

NameError: name 'data_with_indicators' is not defined

In [7]:
"""
Functions for adding ichimoku specific signals to data
"""

def add_ichimoku_features(df):
    is_price_above_cb_lines = []
    is_price_above_cloud = []
    is_price_inside_cloud = []
    is_price_below_cloud = []
    
    for row in data_with_indicators.itertuples():
        cloud_top = None
        cloud_bottom = None  
        if row.trend_visual_ichimoku_a >= row.trend_visual_ichimoku_b:
            cloud_top = row.trend_visual_ichimoku_a
            cloud_bottom = row.trend_visual_ichimoku_b
        else:
            cloud_top = row.trend_visual_ichimoku_b
            cloud_bottom = row.trend_visual_ichimoku_a
            
        if pd.isna(row.trend_ichimoku_conv) or pd.isna(row.trend_ichimoku_base):
            is_price_above_cb_lines.append(None)
            is_price_above_cloud.append(None)
            is_price_inside_cloud.append(None)
            is_price_below_cloud.append(None)
        else:
            if row.Close > row.trend_ichimoku_conv and row.Close > row.trend_ichimoku_base:
                is_price_above_cb_lines.append(True)
            else:
                is_price_above_cb_lines.append(False)
                
            if row.Close > cloud_bottom and row.Close < cloud_top:
                is_price_inside_cloud.append(True)
                is_price_above_cloud.append(False)
                is_price_below_cloud.append(False)
            else:
                is_price_inside_cloud.append(False)
                if row.Close <= cloud_bottom:
                    is_price_above_cloud.append(False)
                    is_price_below_cloud.append(True)
                else:
                    is_price_above_cloud.append(True)
                    is_price_below_cloud.append(False)
        
    df['is_price_above_cb_lines'] = is_price_above_cb_lines
    df['is_price_above_cloud'] = is_price_above_cloud
    df['is_price_inside_cloud'] = is_price_inside_cloud
    df['is_price_below_cloud'] = is_price_below_cloud

    

add_ichimoku_features(data_with_indicators)
data_with_indicators.tail(15)

"""
Function Idea:  When computing the output for a given test input, in other words when deciding if it was best
                to buy or sell at this point, create a function that takes a "max_loss" kinda arg, so if you
                lose more than max_loss...
"""

NameError: name 'data_with_indicators' is not defined